In [1]:
"""
TODO:
1. add previous n_bookings as a feature
2. add number rooms booked previously as average
"""

'\nTODO:\n1. add previous n_bookings as a feature\n2. add number rooms booked previously as average\n'

In [2]:
import pandas as pd
import lightgbm
import category_encoders as ce
from sklearn.preprocessing import CategoricalEncoder,OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import datetime

In [3]:
train = pd.read_csv("train_5CLrC8b/train.csv")
test = pd.read_csv("test.csv")
sample_submission = pd.read_csv("sample_submission.csv")

In [4]:
concated = pd.concat([train,test],ignore_index=True)

/home/prashant/miniconda2/envs/p3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [5]:
concated['booking_date'] = pd.to_datetime(concated['booking_date'],dayfirst=True)
concated['checkin_date'] = pd.to_datetime(concated['checkin_date'],dayfirst=True)
concated['checkout_date'] = pd.to_datetime(concated['checkout_date'],dayfirst=True)

In [6]:
concated['booking_month'] = concated['booking_date'].apply(lambda x:x.month)
concated['booking_year'] = concated['booking_date'].apply(lambda x:x.year)
concated['booking_day'] = concated['booking_date'].apply(lambda x:x.day)

concated['checkin_month'] = concated['checkin_date'].apply(lambda x:x.month)
concated['checkin_year'] = concated['checkin_date'].apply(lambda x:x.year)
concated['checkin_day'] = concated['checkin_date'].apply(lambda x:x.day)

concated['checkout_month'] = concated['checkout_date'].apply(lambda x:x.month)
concated['checkout_year'] = concated['checkout_date'].apply(lambda x:x.year)
concated['checkout_day'] = concated['checkout_date'].apply(lambda x:x.day)

In [7]:
concated['booking_dayofweek'] = concated['booking_date'].apply(lambda x:x.dayofweek)

concated['checkin_dayofweek'] = concated['checkin_date'].apply(lambda x:x.dayofweek)

concated['checkout_dayofweek'] = concated['checkout_date'].apply(lambda x:x.dayofweek)

In [8]:
concated['season_holidayed_code_is_na'] = concated.season_holidayed_code.apply(
    lambda x: 1 if pd.isna(x) else 0)
concated['state_code_residence_is_na'] = concated.state_code_residence.apply(
    lambda x: 1 if pd.isna(x) else 0)
concated['season_holidayed_code'] = concated.season_holidayed_code.apply(
    lambda x: 0 if pd.isna(x) else x)
concated['state_code_residence'] = concated.state_code_residence.apply(
    lambda x: 0 if pd.isna(x) else x)

In [9]:
concated['actual_days_booked'] = concated.apply(
    lambda x: (x['checkout_date'] - x['checkin_date']).days, axis=1)

In [10]:
concated['n_rooms_booked'] = concated.apply(
    lambda x: x['roomnights'] / x['actual_days_booked'], axis=1)

In [11]:
concated['person_per_room'] = concated.apply(
    lambda x: (x['numberofadults'] + x['numberofchildren']) / x['n_rooms_booked'] if x['n_rooms_booked'] else 0,
    axis=1)

In [12]:
def count_weekend(row):
    checkin_date = row['checkin_date']
    actual_days_booked = row['actual_days_booked']
    stayed_in_weekend = 0
    for i in range(actual_days_booked):
        new_day = (checkin_date + datetime.timedelta(days=i)).day_name()
        if new_day == "Saturday" or new_day == "Friday" or new_day == "Sunday":
            stayed_in_weekend += 1
    return stayed_in_weekend

In [13]:
concated['n_weekend_days'] = concated.apply(count_weekend,axis=1)

In [14]:
concated['checkin_month']
seasons = []
for i in list(concated['checkin_month']):
    if i >=1 and i<4:
        seasons.append('w')
    if i>=4 and i<8:
        seasons.append('q')
    if i>=8 and i<11:
        seasons.append('e')
    if i>=11 and i<=12:
        seasons.append('t')
concated['ses']= seasons

In [15]:
concated.drop(['reservation_id','booking_date','checkin_date','checkout_date','memberid'],axis=1,inplace=True)

In [17]:
concated.head()

,amount_spent_per_room_night_scaled,booking_type_code,channel_code,cluster_code,main_product_code,member_age_buckets,numberofadults,numberofchildren,persontravellingid,reservationstatusid_code,...,booking_dayofweek,checkin_dayofweek,checkout_dayofweek,season_holidayed_code_is_na,state_code_residence_is_na,actual_days_booked,n_rooms_booked,person_per_room,n_weekend_days,ses
0,7.706428,1,3,F,1,F,2,0,46,C,...,3,3,4,0,0,1,1.0,2.0,0,q
1,6.662563,1,1,F,1,F,2,0,46,A,...,4,5,3,0,0,5,1.0,2.0,2,q
2,7.871602,1,1,E,1,F,2,0,47,A,...,2,6,3,0,0,4,1.0,2.0,1,w
3,5.344943,1,1,D,1,F,2,2,46,A,...,5,3,1,0,0,5,1.0,4.0,3,q
4,7.059346,1,1,D,1,F,2,0,46,A,...,2,0,5,0,0,5,1.0,2.0,1,t


In [18]:

l=['member_age_buckets','cluster_code','reservationstatusid_code','resort_id','ses']

encoder = ce.target_encoder.TargetEncoder(l)
concated = encoder.fit_transform(concated[l],concated['amount_spent_per_room_night_scaled'])

#encoder = OrdinalEncoder(dtype=np.int32,)
#concated[['memberh_age_buckets','cluster_code','reservationstatusid_code','resort_id']] = encoder.fit_transform(
#    concated[['member_age_buckets','cluster_code','reservationstatusid_code','resort_id']]
#)

In [19]:
concated

,member_age_buckets,cluster_code,reservationstatusid_code,resort_id,ses
0,7.717617,7.712956,7.661260,7.671135,7.763036
1,7.717617,7.712956,7.720382,7.598911,7.763036
2,7.717617,7.780266,7.720382,8.046202,7.642112
3,7.717617,7.686315,7.720382,7.704762,7.763036
4,7.717617,7.686315,7.720382,7.704762,7.760177
5,7.717617,7.712956,7.720382,7.817499,7.642112
6,7.717617,7.701098,7.720382,7.552690,7.642112
7,7.717617,7.712956,7.720382,7.817499,7.723794
8,7.717617,7.701098,7.720382,7.525003,7.723794
9,7.717617,7.780266,7.720382,8.046202,7.760177


In [17]:
# concated.to_csv("./concat_train_test_feat_engg.csv",index=False)
from sklearn.decomposition import PCA
x_ = concated
x_pca_data = x_.drop(['amount_spent_per_room_night_scaled'],axis=1)
pca = PCA(n_components=20)
pca_ = pca.fit_transform(x_pca_data)


In [18]:
new_df = pd.DataFrame(pca_)
new_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-163.956243,-14.293459,-8.472625,-0.481230,-4.035183,-0.129882,-1.727467,-0.500833,-3.570690,-0.912192,-2.215644,0.510533,0.233217,0.836858,-1.423485,-0.194849,1.124141,-1.468723,-0.176304,-0.729728
1,-163.953149,-3.601034,9.639423,-3.742009,-4.914214,-0.219854,-0.204419,-3.793815,1.031361,1.645473,2.035944,-2.653559,1.106201,2.482686,2.606163,0.029395,-0.017914,-1.262802,-0.833819,-0.918949
2,-162.951244,-18.647048,14.193956,-3.198340,-7.358501,-0.077752,-4.197991,-2.101208,0.568684,1.940260,2.629321,-0.882918,1.653821,3.767097,0.939313,0.809558,-0.070410,-0.955601,-1.350147,1.199884
3,-163.957875,-2.836942,-11.412841,-3.218959,-1.037633,-0.337947,-3.083263,-0.896334,0.767392,1.792591,1.916296,-3.133963,-1.353378,0.341069,3.465606,0.211138,-0.142311,-0.688546,-1.388279,-0.777836
4,-163.958030,1.490253,-11.341202,-3.282287,8.303081,-0.239686,-2.972572,-0.655643,0.820047,-3.738684,2.072399,-0.461266,-0.672184,1.661802,2.905237,-0.863487,-0.220713,-1.129495,-0.848953,-0.493182


In [ ]:
#concated.drop(['intercept'],axis=1,inplace=True)
#concated.head()

,intercept,amount_spent_per_room_night_scaled,booking_type_code,channel_code,cluster_code_0,cluster_code_1,cluster_code_2,cluster_code_3,cluster_code_4,main_product_code,...,checkout_dayofweek,season_holidayed_code_is_na,state_code_residence_is_na,actual_days_booked,n_rooms_booked,person_per_room,n_weekend_days,ses_0,ses_1,ses_2
0,1,7.706428,1,3,-0.597614,0.545545,-0.372678,0.188982,-0.062994,1,...,4,0,0,1,1.0,2.0,0,-0.670820,0.5,-0.223607
1,1,6.662563,1,1,-0.597614,0.545545,-0.372678,0.188982,-0.062994,1,...,3,0,0,5,1.0,2.0,2,-0.670820,0.5,-0.223607
2,1,7.871602,1,1,-0.358569,-0.109109,0.521749,-0.566947,0.314970,1,...,3,0,0,4,1.0,2.0,1,-0.223607,-0.5,0.670820
3,1,5.344943,1,1,-0.119523,-0.436436,0.298142,0.377964,-0.629941,1,...,1,0,0,5,1.0,4.0,3,-0.670820,0.5,-0.223607
4,1,7.059346,1,1,-0.119523,-0.436436,0.298142,0.377964,-0.629941,1,...,5,0,0,5,1.0,2.0,1,0.223607,-0.5,-0.670820


In [20]:
train = new_df[:len(train)]
test = new_df[-len(test):]
y = concated[:len(train)]

NameError: name 'new_df' is not defined

In [23]:
all_predictions = []
all_mse = []
X = train#.drop(['amount_spent_per_room_night_scaled'], axis=1)
Y = y['amount_spent_per_room_night_scaled']
for i in range(10):
    print(i)
    trainX, testX, trainY, testY = train_test_split(X, Y,test_size=0.15)
    lgb_model = lightgbm.LGBMRegressor(n_estimators=1000)
    lgb_model.fit(
        trainX,
        trainY,
        eval_set=(testX, testY),
        eval_metric='rmse',
        early_stopping_rounds=100,verbose = 0 
    )
    predictions = lgb_model.predict(test, axis=1)
    test_pred = lgb_model.predict(testX)
    from sklearn.metrics import mean_squared_error
    from math import sqrt
    all_mse.append(sqrt(mean_squared_error(testY, test_pred)))
    all_predictions.append(predictions)

0
1
2
3
4
5
6
7
8
9


In [24]:
internal_pred = np.stack(internal_test_pred,axis=1).mean(axis=1)
final_predictions = np.stack(all_predictions,axis=1).mean(axis=1)

In [25]:
sample_submission['amount_spent_per_room_night_scaled'] = final_predictions
sample_submission.to_csv(
    "poly_enc_1__pca_seson_included.csv",
    index=False)

In [26]:
from sklearn.metrics import mean_squared_error
from math import sqrt
sqrt(mean_squared_error(y_test_internal, pred_orig_test))


NameError: name 'y_test_internal' is not defined

In [27]:
np.array(all_mse).mean(axis=0)

0.9928162715396818

In [ ]:
all_mse

In [23]:

from datetime import date 
import holidays 
  
# Select country 
uk_holidays = holidays.India() 
  
# Print all the holidays in UnitedKingdom in year 2018 
for ptr in holidays.India(years = 2018).items(): 
    print(ptr) 


(datetime.date(2018, 1, 1), 'New Year')
(datetime.date(2018, 1, 14), 'Makar Sankranti / Pongal')
(datetime.date(2018, 1, 26), 'Republic Day')
(datetime.date(2018, 8, 15), 'Independence Day')
(datetime.date(2018, 10, 2), 'Gandhi Jayanti')
(datetime.date(2018, 5, 1), 'Labour Day')
(datetime.date(2018, 12, 25), 'Christmas')
